## Import datasets

In [10]:
from pandas import read_csv

print("--------------------")
print("X RANDOM vs O RANDOM:")
rr = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7920779/random_random_1000.csv")
rr["dataset_id"] = "random_vs_random"
print(rr["outcome"].value_counts(normalize=True))

print("--------------------")
print("X RANDOM vs O EXPERT:")
rm = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7920780/random_minimaxab_1000.csv")
rm["dataset_id"] = "random_vs_expert"
print(rm["outcome"].value_counts(normalize=True))

print("--------------------")
print("X EXPERT vs O RANDOM:")
mr = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7920850/minimaxab_random_1000.csv")
mr["dataset_id"] = "expert_vs_random"
print(mr["outcome"].value_counts(normalize=True))

print("--------------------")
print("X EXPERT vs O EXPERT:")
mm = read_csv("https://github.com/s2t2/tic-tac-toe-py/files/7920782/minimaxab_minimaxab_1000.csv")
print(mm["outcome"].value_counts(normalize=True))
mm["dataset_id"] = "expert_vs_expert"


--------------------
X RANDOM vs O RANDOM:
1.0    0.466982
0.0    0.388867
0.5    0.144151
Name: outcome, dtype: float64
--------------------
X RANDOM vs O EXPERT:
1.0    0.376951
0.0    0.376951
0.5    0.246099
Name: outcome, dtype: float64
--------------------
X EXPERT vs O RANDOM:
1.0    0.559233
0.0    0.390592
0.5    0.050174
Name: outcome, dtype: float64
--------------------
X EXPERT vs O EXPERT:
0.5    1.0
Name: outcome, dtype: float64


## Combine Datasets

In [14]:
from pandas import concat

df = concat([rr, rm, mr, mm])

print(df)

print("-------------")
print("OUTCOMES:")
print(df["outcome"].value_counts(normalize=True))

print("-------------")
print("DATASETS:")
print(df["dataset_id"].value_counts(normalize=True))

      game_id  move_id a1 b1 c1 a2 b2 c2 a3 b3 c3  outcome        dataset_id
0           1        1  X  -  -  -  -  -  -  -  -      0.0  random_vs_random
1           1        2  X  -  -  -  -  -  O  -  -      1.0  random_vs_random
2           1        3  X  -  -  X  -  -  O  -  -      0.0  random_vs_random
3           1        4  X  -  -  X  O  -  O  -  -      1.0  random_vs_random
4           1        5  X  -  -  X  O  -  O  X  -      0.0  random_vs_random
...       ...      ... .. .. .. .. .. .. .. .. ..      ...               ...
8995     1000        5  O  X  -  X  O  -  -  -  X      0.5  expert_vs_expert
8996     1000        6  O  X  O  X  O  -  -  -  X      0.5  expert_vs_expert
8997     1000        7  O  X  O  X  O  -  X  -  X      0.5  expert_vs_expert
8998     1000        8  O  X  O  X  O  -  X  O  X      0.5  expert_vs_expert
8999     1000        9  O  X  O  X  O  X  X  O  X      0.5  expert_vs_expert

[29342 rows x 13 columns]
-------------
OUTCOMES:
0.5    0.412549
1.0    0.

In [15]:
df.to_csv("move_values.csv")